<a href="https://colab.research.google.com/github/graceebc9/Arch_Class/blob/main/Classify_SVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning 
import pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 5.3 MB/s eta 0:00:00


In [ ]:

import torchvision.models as models
import torch
import pandas as pd
import os
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import pytorch_lightning as pl
from PIL import Image



from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

####Update variables 

base_dir = '/content/gdrive/MyDrive/' #update file path for HurricaneDamage_Data Folder


svi_data_drive= '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads'

import sys
sys.path.insert(0, base_dir + '/Non_Trads_FOI')

# from src import 


Mounted at /content/gdrive


In [ ]:


from os import listdir
from os.path import isfile, join

mypath ='/content'
svi = [f for f in listdir(mypath) if isfile(join(mypath, f)) if f[-3:] =='jpg' and f[:3] =='gsv']


S = ['apartment_building/outdoor', 'beach_house', 'building_facade', 'chalet', 'church/outdoor', 'cottage', 'courthouse', 'embassy', 'fire_station', 'hangar/outdoor', 'hospital', 'hotel/outdoor', 'house', 'hunting_lodge/outdoor', 'mansion', 'manufactured_home', 'motel', 'office_building', 'palace', 'schoolhouse', 'shed', 'skyscraper', 'synagogue/outdoor', 'tower']

extra = ['cabin/outdoor', 'inn/outdoor']

targets = S + extra

base = '/content/gdrive/MyDrive/Non_Trads_FOI/data/test_svi/SVI_downloads'
base = '/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images'

folders = os.listdir(base)

In [ ]:

# PlacesCNN for scene classification
#
# by Bolei Zhou
# last modified by Bolei Zhou, Dec.27, 2017 with latest pytorch and torchvision (upgrade your torchvision please if there is trn.Resize error)


# th architecture to use
arch = 'resnet18'

# load the pre-trained weights
model_file = '%s_places365.pth.tar' % arch
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()


# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)


In [ ]:

# # # load the test image
# # img_name = '12.jpg'
# # if not os.access(img_name, os.W_OK):
# #     img_url = 'http://places.csail.mit.edu/demo/' + img_name
# #     os.system('wget ' + img_url)

# # img = Image.open(img_name)
# # input_img = V(centre_crop(img).unsqueeze(0))


# # # load the test image
# img_name = 'gsv_395.jpg'
# img = Image.open(img_name)
# input_img = V(centre_crop(img).unsqueeze(0))

# # forward pass
# logit = model.forward(input_img)
# h_x = F.softmax(logit, 1).data.squeeze()
# probs, idx = h_x.sort(0, True)

# print('{} prediction on {}'.format(arch,img_name))
# # output the prediction
# for i in range(0, 5):
#     print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

In [ ]:

!pwd

/content


In [ ]:
facades

['gsv_395.jpg', 'gsv_394.jpg', 'gsv_393.jpg']

In [ ]:

# ### this works for data in folder structure 
# # for folder in folders:
#   if folder != '.DS_Store': 
#     fpath = os.path.join(base, folder)
#     svi_images  = [f for f in listdir(fpath) if isfile(join(fpath, f)) if f[-3:] =='jpg' and f[:3] =='gsv']
#     facades, labels, probs = [], [], []

#     for img_n in svi_images: 
    
#       img_name = os.path.join(fpath, img_n) 
#       # # load the test image
#       img = Image.open(img_name)
#       input_img = V(centre_crop(img).unsqueeze(0))
#       img.show() 
#       # forward pass
#       logit = model.forward(input_img)
#       h_x = F.softmax(logit, 1).data.squeeze()
#       probs, idx = h_x.sort(0, True)

#       # print('{} prediction on {}'.format(arch,img_name))
#       # # output the prediction
#       # for i in range(0, 5):
#       #     print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))


#       if  classes[idx[0]] in targets:
#         facades.append(img_name)
#         labels.append(classes[idx[0]]) 
#         probs.append(probs[0])

In [ ]:

# for full data not in folder structure 
# for folder in folders:

  svi_images  = [f for f in listdir(fpath) if isfile(join(fpath, f)) if f[-3:] =='jpg' and f[:3] =='gsv']
  facades, labels, probs = [], [], []

  for img_n in svi_images: 
  
    img_name = os.path.join(fpath, img_n) 
    # # load the test image
    img = Image.open(img_name)
    input_img = V(centre_crop(img).unsqueeze(0))
    img.show() 
    # forward pass
    logit = model.forward(input_img)
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)

    # print('{} prediction on {}'.format(arch,img_name))
    # # output the prediction
    # for i in range(0, 5):
    #     print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))


    if  classes[idx[0]] in targets:
      facades.append(img_name)
      labels.append(classes[idx[0]]) 
      probs.append(probs[0])

In [ ]:

  # Use the previously classified data for this image
                # facades.append(img_name)
                # labels_list.append(classified_data[img_name]['label'])
                # probs_list.append(classified_data[img_name]['prob'])
                # print('already processed')

In [ ]:
fpath = '/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images'
svi_images = [f for f in os.listdir(fpath) if isfile(join(fpath, f)) and f[-3:] == 'jpg']

In [ ]:

svi_images[0]

'02826b09e0d1596551331d41b5c65fca43853008bb67a069663f170dba473146_51.61190449999999_-0.067911.jpg'

In [ ]:

len(svi_images)

13372

In [ ]:


import os
import pickle
from os import listdir
from os.path import isfile, join
from PIL import Image
import torch
import torch.nn.functional as F
from torch.autograd import Variable as V

In [ ]:

fpath = '/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images'
svi_images = [f for f in os.listdir(fpath) if isfile(join(fpath, f)) and f[-3:] == 'jpg']



facades, labels_list, probs_list = [], [], []

classified_data_file = '/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/full_data_classified.csv'


if os.path.exists(classified_data_file):
    print('Loading prev data')
    
    # # Load previously classified data
    # with open(classified_data_file, 'rb') as f:
    #     classified_data = pickle.load(f)
    classified_data = pd.read_csv(classified_data_file)
    print(len(classified_data))
else:
    print('no prior data found')
    classified_data = pd.DataFrame(columns = ['image_path', 'label', 'prob'])



Loading prev data
13372


In [ ]:

len(classified_data)

13372

In [ ]:


facades, labels, probs = [], [], []
i= 0 
for img_n in svi_images:
    img_name = os.path.join(fpath, img_n)
    if img_name in classified_data['image_path'].tolist():

        continue
    
    else:
      # Load the test image
      print(img_name)
      img = Image.open(img_name)
      input_img = V(centre_crop(img).unsqueeze(0))
      # img.show()
      
      # Forward pass
      logit = model.forward(input_img)
      h_x = F.softmax(logit, 1).data.squeeze()
      probs, idx = h_x.sort(0, True)
      
      
      # label = classes[idx[1]]
      # prob = probs[1]
      top_probs = [float(p) for p in probs[:4]]
      top_labels = [classes[i] for i in idx[:4]]
    

      # new_row = pd.DataFrame({'image_path': img_name, 'label' : label, 'prob': prob.numpy() }, index=[0])
      new_row = pd.DataFrame({'image_path': img_name, 
                              'label1': top_labels[0], 'label2': top_labels[1], 'label3': top_labels[2], 'label4': top_labels[3],
                              'prob1': top_probs[0], 'prob2': top_probs[1], 'prob3': top_probs[2],  'prob4': top_probs[3]} ,
                              index=[0])
      # Save the classified data for this image
      classified_data = pd.concat([new_row, classified_data.loc[:]]).reset_index(drop=True)
      # classified_data.drop(columns = ['Unnamed: 0'], inplace=True)
      if i % 100 == 0:
        classified_data.to_csv(classified_data_file)
      # print('Saved')
    


Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images/7e4e17a758b736939078b7025a7d5a8938cc2ada9332aea3a6e054a59a04bfd8_55.9385991_-3.4091325.jpg
/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images/a9faa8b83cd1595be6b1ad452b0644e5f3ce5d11da1079a49203c628be38cf87_55.893829_-3.3216997.jpg
/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images/83556626fcb02f744f64d133cd06ab814a2c0f0c9de3d01171be1b419614da57_55.98228349999999_-3.385523.jpg
/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images/e9dda61ab156c3a8ed9f950b5a10032e30b0dd7fdc83f9639de5df1f3245924a_55.98215279999999_-3.3854782.jpg
/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images/5bb11c919a0d642eb8416f173727b5ff451627f5710bbe394d7884104c907e41_55.893907_-3.3212252.jpg
/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/street_view_images/d082abe24a7ebf8b6a3080dedfe3b218454f7778e420f76c3137754fe

In [ ]:

facades = []

for img_name in svi: 

  # # load the test image
  img = Image.open(img_name)
  input_img = V(centre_crop(img).unsqueeze(0))
  img.show() 
  # forward pass
  logit = model.forward(input_img)
  h_x = F.softmax(logit, 1).data.squeeze()
  probs, idx = h_x.sort(0, True)

  print('{} prediction on {}'.format(arch,img_name))
  # output the prediction
  for i in range(0, 5):
      print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))


  if  classes[idx[0]] in targets:
    facades.append(img_name)

In [ ]:
classified_data1 = pd.read_csv('classified_1.csv')

In [ ]:

classified_data1

,Unnamed: 0,image_path,label1,label2,label3,prob1,prob2,prob3,label,prob
0,0,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,house,cottage,0.406592,0.216016,0.045117,NaN,NaN
1,1,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,house,motel,0.523646,0.127478,0.044473,NaN,NaN
2,2,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,house,cottage,0.538173,0.211420,0.057074,NaN,NaN
3,3,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,manufactured_home,shed,barn,0.179136,0.134349,0.124075,NaN,NaN
4,4,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,house,cottage,0.354163,0.256538,0.066727,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
8007,8007,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,house,oast_house,cottage,0.259146,0.144715,0.126777,NaN,NaN
8008,8008,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,beach_house,house,oast_house,0.219494,0.214976,0.160420,NaN,NaN
8009,8009,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,industrial_area,residential_neighborhood,hospital,0.412237,0.327644,0.037960,NaN,NaN
8010,8010,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,house,cottage,0.492159,0.219418,0.076281,NaN,NaN


In [ ]:

classified_data1.drop(columns=['Unnamed: 0', 'Unnamed: 0.1']).dropna()

,image_path,label,prob
0,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,garage/outdoor,0.17811641097068787
1,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,0.6070088744163513
2,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,house,0.2647612988948822
3,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,0.4267372488975525
4,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,0.33084481954574585
...,...,...,...
3469,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,tensor(0.1888)
3470,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,inn/outdoor,tensor(0.3847)
3471,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,residential_neighborhood,tensor(0.2233)
3472,/content/gdrive/MyDrive/Non_Trads_FOI/SVI_down...,manufactured_home,tensor(0.6475)


In [ ]:
idd = '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_317.jpg'

if idd in classified_data1['image_path'].tolist():
  print('y')

y


In [ ]:

 with open(classified_data_file, 'rb') as f:
        classified_data = pickle.load(f)

In [ ]:

classified_data

{'/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_317.jpg': {'label': 'inn/outdoor',
  'prob': tensor(0.3249)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_288.jpg': {'label': 'manufactured_home',
  'prob': tensor(0.2488)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_277.jpg': {'label': 'inn/outdoor',
  'prob': tensor(0.5659)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_88.jpg': {'label': 'inn/outdoor',
  'prob': tensor(0.7094)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_539.jpg': {'label': 'inn/outdoor',
  'prob': tensor(0.3883)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_511.jpg': {'label': 'house',
  'prob': tensor(0.3291)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_505.jpg': {'label': 'house',
  'prob': tensor(0.2131)},
 '/content/gdrive/MyDrive/Non_Trads_FOI/SVI_downloads/bisf/gsv_238.jpg': {'label': 'manufactured_home',
  'prob': tensor(0.8430)},
 

In [ ]:

# for img_name in facades:
#   img = Image.open(img_name)
#   input_img = V(centre_crop(img).unsqueeze(0))
#   img.show() 